In [1]:
import bql
import pandas as pd
import ipywidgets as ipw
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
bq = bql.Service()

In [3]:
# Model Class
class Model():
    
    def __init__(self, bq_serv = None):
        
        self.bq = bq_serv
        
        
    def get_price_data(self, ui):
        '''
        Pulls price data for historical chart
        '''
        
        
        fields = {'Price': self.bq.data.px_last().dropna(),
                 '50DMA': self.bq.data.ma(close = self.bq.data.px_last(currency = ui['fx']), ma_period=50).dropna(),
                 'Volume': self.bq.data.px_volume().dropna()}
        
        with_params = { #'fill': 'prev',
                       'currency': ui['fx'],
                       'dates': self.bq.func.range(ui['start_dt'], ui['end_dt'])}
        
        
        req = bql.Request(ui['ticker'], fields, with_params = with_params)
        res = self.bq.execute(req)
        
        
        df = bql.combined_df(res)
        df = df.set_index('DATE')
        df = df.round(2)
        # df.Price = df.Price.round(2)
        
        
        return df
        
       
    def get_ddis_data(self, ui):
        '''
        Pulls yearly aggregate of amount outstanding to create debt distribution chart
        '''
        
        univ = self.bq.univ.bonds(ui['ticker'], issuedby = 'CAST_PARENT_SUBS')
        
        field = {'Amt Outstanding': self.bq.data.amt_outstanding().group(self.bq.data.maturity().year()).sum().znav()}
        
        with_params = {'fill': 'prev',
                       'currency': ui['fx']}
        
        
        req = bql.Request(univ, field, with_params = with_params)
        res = self.bq.execute(req)
        
        
        df = pd.concat([fld.df()[fld.name] for fld in res], axis = 1, sort = False)
        df = df.rename(index={'NullGroup': 'Perp.'})
        
        
        return df
    
    
    def get_des_data(self, ui):
        '''
        Pulls various descriptive data for Overview
        '''
        
        fields = {'Name': self.bq.data.name(),
                  'Mkt Cap': self.bq.data.market_cap(),
                  'Div. Yield': self.bq.data.div_yield().znav(),
                  'PE': self.bq.data.pe_ratio(fpo='1'),
                  'S&P Rating': self.bq.data.credit_rating(),
                  'Moodys Rating': self.bq.data.credit_rating('MOODY'),
                  'Fitch Rating': self.bq.data.credit_rating('FITCH'),
                  'MSCI ESG Rating': self.bq.data.esg_rating('MSCI'),
                  'Bloomberg ESG Score': self.bq.data.esg_score(score_source='BBG')}
        
        with_params = {'fill': 'prev',
                       'currency': ui['fx']}
        
        
        req = bql.Request(ui['ticker'], fields, with_params = with_params)
        res = self.bq.execute(req)
        
        df = pd.concat([fld.df()[fld.name] for fld in res], axis = 1, sort = False)
        
        return df      
    
    
    def get_est_data(self, ui, field):
        '''
        Pulls EPS estimates for Earnings chart
        '''

        # Get field key and value from UI to use in BQL request
        fields = {ui['est']: field,
                  'SD': field.contributor_stats(stat_type='STD')}

        with_params = {'fpt': 'a',
                       'fill': 'prev',
                       'fpo': '1',
                       'dates': self.bq.func.range(ui['start_dt'], ui['end_dt']),
                       'currency': ui['fx'],
                       'act_est_mapping': 'precise',
                       'fs': 'MRC'}

        req = bql.Request(ui['ticker'], fields, with_params = with_params)
        res = self.bq.execute(req)

        df = bql.combined_df(res)
        
        df = df.set_index('AS_OF_DATE')
        df = df.drop(['REVISION_DATE', 'PERIOD_END_DATE', 'CURRENCY'], axis = 1)
        # df = df[ui['est']].apply(lambda x : "{:,}".format(x))
        
        df[ui['est']] = df[ui['est']].abs()
        df['+1SD'] = df[ui['est']] + df['SD']
        df['-1SD'] = df[ui['est']] - df['SD']
        
        df = df.round(2)
        
        
        return df
    
    
    def get_divs_data(self, ui):
        '''
        Pulls historical and forward-looking Dividend Per Share (DPS) for Dividends chart
        '''
        
        field = {'DPS': self.bq.data.headline_dps()}
        with_params = {'fpt': 'a',
                       'fill': 'prev',
                       'fpo': self.bq.func.range('-10', '6'),
                       'currency': ui['fx']}
        
        
        req = bql.Request(ui['ticker'], field, with_params = with_params)
        res = self.bq.execute(req)
        
        df = res[0].df()
        
        df = df.set_index('PERIOD_END_DATE')
        df = df.round(2)
        
        
        return df
    
    
    def get_margins_data(self, ui):
        '''
        Pulls historical margins for Profitability tab
        '''
        
        fields = {'Gross Margin': self.bq.data.gross_profit()/self.bq.data.is_comp_sales(),
                  'Operating Margin': self.bq.data.is_comparable_ebit()/self.bq.data.is_comp_sales(),
                  'EBITDA Margin': self.bq.data.is_comparable_ebitda()/self.bq.data.is_comp_sales(),
                  'Net Margin': self.bq.data.is_comp_net_income_gaap()/self.bq.data.is_comp_sales()}
        
        params = {'fpo': self.bq.func.range('-7', '5'),
                  'fpt': 'a',
                  'act_est_mapping': 'precise',
                  'fs': 'MRC'}
        
        
        req = bql.Request(ui['ticker'], fields, with_params = params)
        res = self.bq.execute(req)
        
        df = bql.combined_df(res)
        df = df.set_index('PERIOD_END_DATE')
        df = df.drop(['CURRENCY', 'AS_OF_DATE', 'REVISION_DATE'], axis=1)
        df = df*100
        df = df.round(2)
        
        return df
        
    
    def chart_price(self, df):
        '''
        Creates Price and Volume chart for the Overview Tab
        '''
        
        # Create the subplot figure
        px_fig = make_subplots(rows = 2, 
                            cols = 1, 
                            shared_xaxes = True,
                            vertical_spacing = 0.05,
                            row_width = [0.3, 0.8])
        
        # Add the individual traces: Price, Moving Average, and Volume
        px_fig.add_trace(go.Scatter(x = df.index, y = df['Price'], name = 'Price'), row = 1, col = 1)
        px_fig.add_trace(go.Scatter(x = df.index, y = df['50DMA'], name = '50DMA'), row = 1, col = 1)
        px_fig.add_trace(go.Bar(x = df.index, y = df['Volume'], name = 'Volume'), row = 2, col = 1)
            
        
        # Put figure into a Widget container
        px_fig = go.FigureWidget(px_fig)
        

        # Change line colours and add title
        # colours = ['LightBlue', 'Teal', 'Beige']
        px_fig.update_layout(bargap = 0,
                             bargroupgap = 0,
                             colorway = ['LightBlue', 'Teal', 'Lavender'], 
                             title = 'Price Chart',
                             title_x = 0.5)
        
        
        return px_fig
    
    
    def chart_ddis(self, df):
        '''
        Create chart for the Debt Distribution tab
        '''
        
        # Define the traces
        debt_traces = go.Bar(x = [year[:4] for year in list(df.index)],
                             y = df['Amt Outstanding'])
        
        # Create the chart
        debt_fig = go.FigureWidget(data = debt_traces)
        
        # Change line colours and add title
        debt_fig.update_layout(colorway = ['Aqua'], 
                               title = 'Debt Distribution',
                               title_x = 0.5)
        
        
        return debt_fig
    
    
    def chart_est(self, df):
        '''
        Create chart for the Estimates tab
        '''
        
        # Define the traces
        est_traces = [go.Scatter(x = df.index,
                                 y = df[col],
                                 name = col) 
                      for col in df.columns if col not in ['SD']]
        
        est_fig = go.FigureWidget(data = est_traces)
        
        # Change line colours and add title
        est_fig.update_layout(colorway = ['Teal','LightBlue', 'Aqua'])
        
        
        return est_fig
    
    
    def chart_divs(self, df):
        '''
        Create chart for the Dividends tab
        '''
        
        # Define the traces
        divs_traces = go.Scatter(x = df.index,
                                 y = df['DPS'],
                                 name = 'DPS')
        
        divs_fig = go.FigureWidget(data = divs_traces)
        
        # Change line colours and add title
        divs_fig.update_layout(colorway = ['Teal'],
                               title = 'Annual Dividends Per Share - Historical and Consensus',
                               title_x = 0.5)
        
        # Add vertical line as of today to mark separation between actual data and estiamtes
        divs_fig.add_vline(x = datetime.date.today().strftime("%Y-%m-%d"))
        
        
        return divs_fig
    
    
    def chart_margins(self, df):
        '''
        Create chart for the Margins tab
        '''
        
        # Define the traces
        margin_traces = traces = [go.Scatter(x=df.index, y=df[col], name=col) for col in df]
                
        margin_fig = go.FigureWidget(data = margin_traces)
        
        # Change line colours and add title
        colors = ['LightCyan', 'LightBlue', 'LavenderBlush', 'Lavender']
        margin_fig.update_layout(colorway = ['Azure', 'Cyan', 'DarkCyan', 'White'],
                                 title = 'Margin Analysis',
                                 title_x = 0.5)
        
        margin_fig.add_vline(x = datetime.date.today().strftime("%Y-%m-%d"))
        
        
        return margin_fig
    

In [4]:
# View Class
class View(ipw.VBox):
    
    def __init__(self, controller = None):
        
        super().__init__() 
        self.ctrl = controller # Instantiate controller
        self.widgets = {} # Create empty dict for widgets
        self._build_view() # Build the UI
        
        
    def _build_view(self): 
        
        # Instantiate Start View
        self.widgets['start_view'] = StartView(controller = self.ctrl)        
        

        # Build startup view
        self.children = [self.widgets['start_view']]   
                 
            
    def set_results(self, px_fig = None, debt_fig = None, est_fig = None, divs_fig = None, margins_fig = None):
        
        self.widgets['results_view'] = ResultsView(px_fig, debt_fig, est_fig, divs_fig, margins_fig)
        self.children = [self.widgets['start_view'], self.widgets['results_view']]
                       
            
    def set_error_msg(self,error):
        err_widget = ipw.HTML(f'<p style="color:red;" >{error}</p>')
        self.children = [self.widgets['start_view'], err_widget]
           

In [5]:
class StartView(ipw.VBox):
    
    def __init__(self, controller = None):
        super().__init__()
        self.ctrl = controller
        self.widgets = {}
        self.fields = {}
        self._build_view()
        
        
    def _build_view(self):
        '''
        Create startup view with input widgets and default values
        '''
                
        # Layouts
        lbl_layout = {'width': '70px'}
        input_layout = {'width': '160px'}
        
        # Fields for Estimates analysis
        self.fields['CapEx'] = bq.data.headline_capex()
        self.fields['DPS'] = bq.data.headline_dps()
        self.fields['EPS'] = bq.data.is_comp_eps_gaap()
        self.fields['EBITDA'] = bq.data.is_comparable_ebitda()
        self.fields['FCF'] = bq.data.headline_fcf()
        self.fields['Gross Margin'] = bq.data.is_comp_gross_margin_percentage()
        self.fields['Net Income'] = bq.data.is_comp_net_income_gaap()
        self.fields['Operating Income'] = bq.data.is_comparable_ebit()
        self.fields['Revenue'] = bq.data.is_comp_sales()
        
        # Currency Options
        currencies = ['ARS', 'AUD', 'BRL', 'CAD', 'CHF', 
                      'CNY', 'EUR', 'GBP', 'HKD', 'IDR', 
                      'INR', 'JPY', 'KRW', 'MXN', 'RUB', 
                      'SAR', 'SGD', 'TRY', 'USD', 'ZAR']
        
        # Labels
        self.widgets['ticker_lbl'] = ipw.Label(value = 'Ticker', layout = lbl_layout)
        self.widgets['start_dt_lbl'] = ipw.Label(value = 'Start Date', layout = lbl_layout)
        self.widgets['end_dt_lbl'] = ipw.Label(value = 'End Date', layout = lbl_layout)
        self.widgets['est_lbl'] = ipw.Label(value = 'Est. Field', layout = lbl_layout)
        self.widgets['fx_lbl'] = ipw.Label(value = 'Currency', layout = lbl_layout)
        
        # Input Widgets
        self.widgets['ticker'] = ipw.Text(value = 'AAPL US Equity', layout = input_layout)
        self.widgets['start_dt'] = ipw.DatePicker(value = datetime.date.today() - relativedelta(years=5), layout = input_layout)
        self.widgets['end_dt'] = ipw.DatePicker(value = datetime.date.today(), layout = input_layout)
        self.widgets['est'] = ipw.Dropdown(value = 'EPS', options = list(self.fields.keys()), layout = input_layout)
        self.widgets['fx'] = ipw.Dropdown(value = 'EUR', options = currencies, layout = input_layout)
        
        # Controls
        self.widgets['controls'] = ipw.VBox([ipw.HBox([self.widgets['ticker_lbl'], self.widgets['ticker']]),
                                             ipw.HBox([self.widgets['start_dt_lbl'], self.widgets['start_dt']]),
                                             ipw.HBox([self.widgets['end_dt_lbl'], self.widgets['end_dt']]),
                                             ipw.HBox([self.widgets['est_lbl'], self.widgets['est']]),
                                             ipw.HBox([self.widgets['fx_lbl'], self.widgets['fx']])])
        
        # Button
        self.widgets['btn'] = ipw.Button(description = 'Get Data', button_style = 'success', layout = {'width': '160px'})
        self.widgets['btn'].on_click(self.ctrl.run)
        self.widgets['btn_view'] = ipw.HBox([self.widgets['btn']], layout = {'margin': '10px 0px 10px 75px'})
        
        # Widgets for "in progress" view
        spinner = ipw.HTML('''<i class="fa fa-spinner fa-spin" style="font-size:24px"></i>''')
        lbl_update = ipw.Label('Requesting data...')
        self.widgets['update_view'] = ipw.HBox([spinner, lbl_update], layout = {'visibility': 'hidden'})
        
        
        
        # Input View
        self.widgets['input_view'] = ipw.Tab([ipw.VBox([self.widgets['controls'],
                                                        self.widgets['btn_view'],
                                                        # self.widgets['update_view']
                                                       ])])
        
        self.widgets['input_view'].set_title(0, 'Controls')
        self.widgets['input_view'].layout = {'width': '800px'}
        
        # Description View
        # self.widgets['des_view'] = ipw.VBox()

                
        # self.children = [self.widgets['input_view'], self.widgets['des_view']]
        self.children = [self.widgets['input_view']]
        
        
    def show_spinner(self, show):
        '''
        Controls if the spinner is visible or not
        '''
        
        if show:
            self.widgets['update_view'].layout.visibility = 'visible' 
        else: 
            self.widgets['update_view'].layout.visibility = 'hidden'
        
        
    def read_ui(self):
        '''
        Reads user inputs and stores them in a dictionary
        '''
        
        ui = {'ticker': self.widgets['ticker'].value,
              'start_dt': self.widgets['start_dt'].value,
              'end_dt': self.widgets['end_dt'].value,
              'est': self.widgets['est'].label,
              'est_fld': self.widgets['est'].value,
              'fx': self.widgets['fx'].value}
        
        
        return ui
       

In [6]:
class ResultsView(ipw.Tab):
    
    def __init__(self, px_fig = None, debt_fig = None, est_fig = None, divs_fig = None, margins_fig = None):
        super().__init__()
        self.px_fig = px_fig
        self.debt_fig = debt_fig
        self.est_fig = est_fig
        self.divs_fig = divs_fig
        self.margins_fig = margins_fig
        self.widgets = {}
        self._build_view()
    
    
    def _build_view(self):
        
 
        # Add results Widgets to main widgets dictionary
        self.widgets['px_chart'] = self.px_fig
        self.widgets['ddis_chart'] = self.debt_fig
        self.widgets['est_chart'] = self.est_fig
        self.widgets['divs_chart'] = self.divs_fig
        self.widgets['margins_chart'] = self.margins_fig
        
        
        # Create Tabs to display results
        tab_titles = ['Overview', 'Estimates', 'Margins', 'Dividends', 'Debt Distribution']
        
        # Assign results to the Results View
        self.children = [self.widgets['px_chart'], 
                         self.widgets['est_chart'],
                         self.widgets['margins_chart'],
                         self.widgets['divs_chart'], 
                         self.widgets['ddis_chart']]
        
        self.layout = {'width': '800px'}
        
        # Apply Titles to Tabs
        for index, title in enumerate(tab_titles):
            self.set_title(index, title) 
            

In [7]:
# Controller Class
class Controller():
    
    def __init__(self, bq_serv = None):
        
        self.bq = bq_serv
        self.model = Model(bq_serv = bq) # Instantiate the model class to get data
        self.view = View(controller = self) # Instantiate the view classes to manipulate the GUI
        self.sv = StartView(controller = self)
        
        
        
    def show(self):
        
        return self.view # Displays the app when a Controller object is instantiated
        
        
    def run(self, *args):
        '''
        Main "run" function which gets called when user clicks the Get Data button
        '''
        
        # Layouts to apply to all charts
        layouts = {'template': 'plotly_dark',
                   'plot_bgcolor': 'rgba(33,33,33,33)',
                   'paper_bgcolor': 'rgba(33,33,33,33)',
                   'height': 450,
                   'legend_x': 0.01, 
                   'legend_y': -0.05,
                   'legend': {'orientation': 'h'},
                   'width': 700}
        
        # Update view to reflect data being fetched
        self.sv.show_spinner(True)
        
                
        try:
            ui = self.view.widgets['start_view'].read_ui()  # Get user inputs from UI
            est_field = self.sv.fields[ui['est']] # Get estimate field from UI

            # Create the various dataframes needed to generate charts
            price_df = self.model.get_price_data(ui)
            debt_df = self.model.get_ddis_data(ui)
            est_df = self.model.get_est_data(ui, est_field)
            divs_df = self.model.get_divs_data(ui)
            margins_df = self.model.get_margins_data(ui)
            
            # Create corresponding charts
            px_fig = self.model.chart_price(price_df)
            debt_fig = self.model.chart_ddis(debt_df)
            est_fig = self.model.chart_est(est_df)
            divs_fig = self.model.chart_divs(divs_df)
            margins_fig = self.model.chart_margins(margins_df)
            
            # Apply title to Estimates chart (doing it here as we need the selected field from the view)
            est_fig.update_layout(title = 'Next Fiscal Year Estimates - ' + ui['est'], title_x = 0.5)
            
            # Apply layout to all charts
            figures = [px_fig, debt_fig, est_fig, divs_fig, margins_fig]
            [fig.update_layout(layouts) for fig in figures]
                
            # Create the Results View          
            self.view.set_results(px_fig, debt_fig, est_fig, divs_fig, margins_fig)
            
        except Exception as e:
            self.view.set_error_msg(str(e))
        
        
        self.sv.show_spinner(False)
              

In [8]:
app = Controller(bq_serv = bq)

In [9]:
app.show()

View(children=(StartView(children=(Tab(children=(VBox(children=(VBox(children=(HBox(children=(Label(value='Tic…